In [1]:
import scipy
scipy.__version__

'0.15.1'

In [2]:
import numpy as np
import scipy.optimize

    Minimise -1*x[0]-2*x[1]-3*x[2]
    subject to 5*x[0]+6*x[1]+7*x[2] <= 100
               5*x[0]+6*x[1]+7*x[2] <= 101

In [3]:
res = scipy.optimize.linprog(c=[-1,-2,-3], A_ub = [ [5,6,7], [8,9,10] ], b_ub = [100,101])
print(res)

   slack: array([ 29.3,   0. ])
 message: 'Optimization terminated successfully.'
       x: array([  0. ,   0. ,  10.1])
     fun: -30.299999999999997
     nit: 1
 success: True
  status: 0


In [5]:
def Solve_old(x, y, full=False):
    """Solves the matrix equation x*t = 1, y*t = 1
    and also t_1 - T <= 0, ..., t_n - T <=0,
    and returns minimal possible T."""
    A = [ [0] * len(x) + [-1] for _ in range(len(x)) ]
    for i in range(len(x)):
        A[i][i] = 1
    c = [0]*len(x) + [1]
    b = [0]*len(x)
    Aeq = [x+[0], y+[0]]
    if full:
        print("Solving {}*t = [1,1]".format(Aeq))
        print("{}*t <= {}".format(A, b))
        print("Minimising {}*t".format(c))
    #res = scipy.optimize.linprog(c=c, A_ub = A, b_ub = b, A_eq = Aeq, b_eq = [1,1] )
    res = scipy.optimize.linprog(c, A, b, Aeq, [1.0, 1.0], bounds = (-1e-8, None))
    if full: return res
    return res.fun, res.success

def Solve(x, y, full=False):
    """Solves the matrix equation x*t = 1, y*t = 1
    and also t_1 - T <= 0, ..., t_n - T <=0,
    and returns minimal possible T."""
    A = [ [0] * len(x) + [-1] for _ in range(len(x)) ]
    for i in range(len(x)):
        A[i][i] = 1
    A.extend( [0]*(len(x)+1) for _ in range(len(x)) )
    for i in range(len(x)):
        A[i+len(x)][i] = -1
    c = [0]*len(x) + [1]
    b = [0]*len(x)*2
    Aeq = [x+[0], y+[0]]
    print(A,b)
    if full:
        print("Solving {}*t = [1,1]".format(Aeq))
        print("{}*t <= {}".format(A, b))
        print("Minimising {}*t".format(c))
    #res = scipy.optimize.linprog(c=c, A_ub = A, b_ub = b, A_eq = Aeq, b_eq = [1,1] )
    res = scipy.optimize.linprog(c, A, b, Aeq, [1.0, 1.0], bounds = (-1, None))
    if full: return res
    return res.fun, res.success

In [6]:
with open("B-small-practice.in") as f:
    with open("B-small.out") as g:
        numCases = int(next(f))
        for case in range(1, numCases+1):
            data = next(f).split()
            N = int(data[0])
            V, X = float(data[1]), float(data[2])
            R, C = zip(*[ [float(x) for x in next(f).split()] for _ in range(N)])
            #print("V={}, X={}".format(V,X))
            #print("R={}, C={}".format(R,C))
            x = [r/V for r in R]
            y = [r*c/(V*X) for r, c in zip(R,C) ]
            #print("==> x={}, y={}".format(x,y))
            soln = Solve(x, y)
            shouldbe = next(g).rstrip()
            shouldbe = shouldbe[shouldbe.find(":")+2:]
            if soln[1]:
                print("Case #{}: {} -- {}".format(case, soln[0], abs(soln[0]-float(shouldbe))))
            else:
                if shouldbe != "IMPOSSIBLE":
                    raise Exception("Should be {} but I think IMPOSSIBLE".format(shouldbe))
                print("Case #{}: IMPOSSIBLE".format(case))

[[1, 0, -1], [0, 1, -1], [-1, 0, 0], [0, -1, 0]] [0, 0, 0, 0]
Case #1: 1.9970881863560779 -- 2.220446049250313e-15
[[1, 0, -1], [0, 1, -1], [-1, 0, 0], [0, -1, 0]] [0, 0, 0, 0]


Exception: Should be 5.000025000125E-06 but I think IMPOSSIBLE

In [8]:
with open("B-small-practice.in") as f:
    numCases = int(next(f))
    for case in range(1, numCases+1):
        data = next(f).split()
        N = int(data[0])
        V, X = float(data[1]), float(data[2])
        R, C = zip(*[ [float(x) for x in next(f).split()] for _ in range(N)])
        x = [r/V for r in R]
        y = [r*c/(V*X) for r, c in zip(R,C) ]
        if case == 2:
            print("V={}, X={}".format(V,X))
            print("R={}, C={}".format(R,C))
            print("==> x={}, y={}".format(x,y))
            print(Solve(x, y, True))
            break

V=0.0005, X=83.5067
R=(99.9998, 99.9995), C=(60.2114, 83.5067)
==> x=[199999.59999999998, 199999.0], y=[144207.06261222152, 199999.0]
[[1, 0, -1], [0, 1, -1], [-1, 0, 0], [0, -1, 0]] [0, 0, 0, 0]
Solving [[199999.59999999998, 199999.0, 0], [144207.06261222152, 199999.0, 0]]*t = [1,1]
[[1, 0, -1], [0, 1, -1], [-1, 0, 0], [0, -1, 0]]*t <= [0, 0, 0, 0]
Minimising [0, 0, 1]*t
     nit: 4
  status: 2
 message: 'Optimzation failed. Unable to find a feasible starting point.'
 success: False
     fun: -5.8207660913467407e-11
       x: nan


In [9]:
RR = [r for r in R]
CR = [c*r for c, r in zip(C,R)]
res = scipy.optimize.linprog([0,0,1], [[1,0,-1],[0,1,-1],[-1,0,0],[0,-1,0]], [0,0,0,0],
                             [RR+[0],CR+[0]], [V, X*V], bounds = (-1e-8, None))
res

     nit: 3
  status: 0
 message: 'Optimization terminated successfully.'
 success: True
   slack: array([  5.00002500e-06,   0.00000000e+00,   1.00000000e-08,
         5.01002500e-06])
     fun: 5.0000250001249988e-06
       x: array([  3.67929936e-21,   5.00002500e-06,   5.00002500e-06])

In [102]:
RR, CR, V, X*V

([0.0001, 99.9999], [0.008839830000000002, 8839.83116016], 100.0, 8839.83)

In [105]:
RR[0] * 1e6- V, CR[0]*1e6 - X*V

(0.0, 1.8189894035458565e-12)

## Recast in the new way ##

A different way to specify the optimisation problem is to:

$$ \text{Maximise}\quad \sum_i R_i'
\quad\text{subject to}\quad \sum_i R_i'(C_i-X) = 0, \quad 0\leq R_i'\leq R_i.  $$

(And then $T = V / \sum_i R_i'$.)

In [20]:
with open("B-large-practice.in") as f:
    with open("B-large.out") as g:
        numCases = int(next(f))
        for case in range(1, numCases+1):
            data = next(f).split()
            N = int(data[0])
            V, X = float(data[1]), float(data[2])
            R, C = zip(*[ [float(x) for x in next(f).split()] for _ in range(N)])
            
            A_ub = [ [0]*len(R) for _ in range(len(R)) ]
            for i in range(len(R)):
                A_ub[i][i] = 1
            res = scipy.optimize.linprog([-1]*len(R), A_eq=[[c-X for c in C]], b_eq=[0], A_ub = A_ub, b_ub = R)
            if not res.success:
                raise Exception("Optimisation failed!")
            if sum(res.x) == 0:
                print("IMPOSSIBLE", next(g).strip())
            else:
                T = V / sum(res.x)
                print(T, next(g).strip())
            #if case == 14: break

0.998499419715 Case #1: 0.9984994197151571
3.53447334411 Case #2: 3.534473344112177
0.039407411796 Case #3: 0.039407411796042756
7.53493991258e-09 Case #4: 7.534939912582608e-09
0.01588305335 Case #5: 0.01588305335002975
0.432927629729 Case #6: 0.4329276297289952
0.0189714987169 Case #7: 0.018971498716944736
1.42803489546 Case #8: 1.4280348954607054
18.9753320683 Case #9: 18.975332068311197
0.0201052061035 Case #10: 0.020105206103466458
0.399047074568 Case #11: 0.3990470745683413
5.28977361471e-10 Case #12: 5.289773614711203e-10
IMPOSSIBLE Case #13: IMPOSSIBLE
5.50398335893 Case #14: 5.503983358927176
IMPOSSIBLE Case #15: IMPOSSIBLE
49126760.0511 Case #16: 49126760.051128134
0.312859275902 Case #17: 0.3128592759020815
0.172514700671 Case #18: 0.17251470067075433
24999998.5 Case #19: 24999998.499999996
IMPOSSIBLE Case #20: IMPOSSIBLE
IMPOSSIBLE Case #21: IMPOSSIBLE
1000000.0 Case #22: 1000000.0
0.0480617638134 Case #23: 0.04806176381337739
0.0529388195326 Case #24: 0.05293881953258677
0